# <p style="text-align:center;">Tensorflow - III</p>
---
*<p style="text-align:right;">Reference: Tensorflow Official Docs</p>*


In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Introduction to Variables

A TensorFlow variable is the recommended way to represent shared, persistent state your program manipulates. This guide covers how to create, update, and manage instances of `tf.Variable` in TensorFlow.

Variables are created and tracked via the `tf.Variable` class. A `tf.Variable` represents a tensor whose value can be changed by running ops on it. Specific ops allow you to read and modify the values of this tensor. Higher level libraries like `tf.keras` use `tf.Variable` to store model parameters.

## 1. Create a Variable

To create a variable, provide an initial value. The `tf.Variable` will have the same `dtype` as the initialization value.

In [2]:
my_tensor = tf.constant([[1.,2.],[3.,4.]])
my_variable = tf.Variable(my_tensor)

# Variables can be all kinds of types, just like tensors
bool_variable = tf.Variable([False, False, False, True])
complex_variable = tf.Variable([5 + 4j, 6 + 1j])

A variable looks and acts like a tensor, and, in fact, is a data structure backed by a `tf.Tensor`. Like tensors, they have a `dtype` and a shape, and can be exported to NumPy.

In [3]:
print("shape: ", my_variable.shape)
print("Dtype: ", my_variable.dtype)
print("As Numpy: ", my_variable.numpy())

shape:  (2, 2)
Dtype:  <dtype: 'float32'>
As Numpy:  [[1. 2.]
 [3. 4.]]


Most tensor operations work on variable as expected, although variables cannot be reshaped.

In [4]:
print("A variable: ", my_variable)
print("\nViewed as a tensor:", tf.convert_to_tensor(my_variable))
print("\n Index of highest value", tf.math.argmax(my_variable))

print("\nCopying and reshaping:",tf.reshape(my_variable,[1,4]))

A variable:  <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

Viewed as a tensor: tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)

 Index of highest value tf.Tensor([1 1], shape=(2,), dtype=int64)

Copying and reshaping: tf.Tensor([[1. 2. 3. 4.]], shape=(1, 4), dtype=float32)


As noted above, variables are backed by tensors. You can reassign the tensor using `tf.Variable.assign`. Calling `assign` does not (usually) allocate a new tensor; instead ,the exiting tensor's memory is reused.

In [5]:
a = tf.Variable([[1,2,3],[4,5,6]])
a.assign([7,8,9])
try:
    a.assign([1,2])
except Exception as e:
    print(f"{type(e).__name__}:{e}")

ValueError: Cannot assign value to variable ' Variable:0': Shape mismatch.The variable shape (2, 3), and the assigned value shape (3,) are incompatible.

In [6]:
a.numpy()

array([[1, 2, 3],
       [4, 5, 6]])

If you use a variable like a tensor in operations, you will usually operate on the backing tensor. Creating new variables from existing variables duplicates the backing tensors. Two variables will not share the same memory

In [7]:
a = tf.Variable([2.0,3.0])
b = tf.Variable(a)
a.assign([5,6])
print(a.numpy())
print(b.numpy())

print(a.assign_add([2,3]).numpy())
print(a.assign_sub([7,9]).numpy())

[5. 6.]
[2. 3.]
[7. 9.]
[0. 0.]


## 2. Lifecycles, naming, and watching

In Python-based TensorFlow, `tf.Variable` instance have the same lifecycle as other Python objects. When there are no references to a variable it is automatically deallocated.

Variables can also be named which can help you track and debug them. You can give two variables the same name.

In [8]:
#create 2 different variables named a and b. They wil have same name but will
# be backed by different tensors
a = tf.Variable(my_tensor, name='Mark')
#new variable with same name but different value
b = tf.Variable(my_tensor + 1, name='Mark')

print(a==b)

tf.Tensor(
[[False False]
 [False False]], shape=(2, 2), dtype=bool)


Variable names are preserved when saving and loading models. By default, variables in models will acquire unique variable names automatically, so you don't need to assign them yourself unless you want to.

Although variables are important for differentiation, some variables will not need to be differentiated. **You can turn off gradients for a variable by setting `trainable` to false at creation**. An example of a variable that would not need gradients is a training step counter.

In [9]:
step_counter = tf.Variable(1, trainable=False)

## 3.  Placing variables and tensors

For better performance, TensorFlow will attempt to place tensors and variables on the fastest device compatible with its `dtype`. This means most variables are placed on a GPU if one is available.

However, you can override this. In this snippet, place a float tensor and a variable on the CPU, even if a GPU is available. By turning on device placement logging (see Setup), you can see where the variable is placed.